# ONS API Data Retrieval (https://covid-api.com/)
## Imports
### Importing necessary libraries & modules

In [1]:
import requests
import pandas as pd
from io import StringIO

## API connectors
### Here I define the functions I will be using to connect to the ONS API & retrieve data

In [2]:
base_url = 'https://api.beta.ons.gov.uk/v1/'

In [3]:
def get_data(url, params=None):
    request_response = requests.get(url, params=params)
    if request_response.status_code == 200:
        return request_response.json()
    else:
        return [request_response.status_code, request_response.text]

In [4]:
def get_available_datasets():
    available_datasets_response = get_data(f"{base_url}datasets")
    
    available_datasets_response = available_datasets_response['items']
    
    available_datasets = {}
    for i, available_dataset in enumerate(available_datasets_response):
        available_datasets[f"{i+1}"] =  available_dataset['id']
        
    return available_datasets, available_datasets_response

In [5]:
def get_data_link(i, available_datasets_response):
    try:
        data_link = available_datasets_response[i-1]['links']['latest_version']['href']
        print(data_link)
        return data_link
    except IndexError:
        print("This is not a valid selection, please choose an appropriate dataset")

In [6]:
def get_data_for_selected_parameter(link):
    link_response = get_data(link)
    csv_link = link_response['downloads']['csv']['href']
    response = requests.get(csv_link)
    dataframe = pd.read_csv(StringIO(response.text))
    return dataframe

## Functions to display available dataset & save data to csv file

In [7]:
def print_dataset(available_datasets):
    for i in range(1, len(available_datasets)):
        print(f"{i}. {available_datasets[str(i)].replace('-', ' ').title()}")

In [8]:
def save_to_csv(file_name, dataframe):
    dataframe.to_csv(file_name, index=False)

## Running the functions to make data selection

In [9]:
available_datasets, available_datasets_response = get_available_datasets()

In [10]:
print_dataset(available_datasets)

1. Wellbeing Quarterly
2. Wellbeing Local Authority
3. Weekly Deaths Region
4. Weekly Deaths Local Authority
5. Weekly Deaths Health Board
6. Weekly Deaths Age Sex
7. Uk Spending On Cards
8. Uk Business By Enterprises And Local Units
9. Traffic Camera Activity
10. Trade
11. Tax Benefits Statistics
12. Suicides In The Uk
13. Sexual Orientation By Region
14. Sexual Orientation By Age And Sex
15. Retail Sales Index Large And Small Businesses
16. Retail Sales Index All Businesses
17. Retail Sales Index
18. Regional Gdp By Year
19. Regional Gdp By Quarter


### Please enter the number of the dataset you would like to select:

In [11]:
selected_dataset = 11

### The below code will save the output to a csv

In [12]:
link = get_data_link(selected_dataset, available_datasets_response)

https://api.beta.ons.gov.uk/v1/datasets/tax-benefits-statistics/editions/time-series/versions/3


In [13]:
dataframe = get_data_for_selected_parameter(link)

In [14]:
file_name = available_datasets[str(selected_dataset)].replace('-', '_')

In [15]:
save_to_csv(f'2_processed_databases/ons_data/{file_name}.csv', dataframe)